In [27]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import SVD
from surprise.model_selection import train_test_split
from surprise import dump
import csv
from surprise import accuracy

In [28]:
# Path to dump file and name
dumpfile = os.path.join('../data/dump/dump_svd_barryspicksdump_file')

beer_pickel_path = os.path.join('../data/dump/beer.pkl')

In [29]:
# Lets load our dump files
predictions,algo = dump.load(dumpfile)

beers_df = pd.read_pickle(beer_pickel_path)

In [30]:
def get_beer_name (beer_raw_id):
    beer_name = beers_df.loc[beers_df.beer_id==beer_raw_id,'name'].values[0]
    return beer_name

def get_beer_raw_id (beer_name):
    beer_raw_id = beers_df.loc[beers_df.name==beer_name,'beer_id'].values[0]
    return beer_raw_id

def get_beer_style (beer_raw_id):
    beer_style = beers_df.loc[beers_df.beer_id==beer_raw_id,'style'].values[0]
    return beer_style

def get_beer_score_mean (beer_raw_id):
    score_mean = beers_df.loc[beers_df.beer_id==beer_raw_id,'score'].values[0]
    return score_mean

def get_beer_neighbors (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                  for inner_id in beer_neighbors)
    return(beer_neighbors)

def get_beer_recc_df (beer_raw_id):
    beer_inner_id = algo.trainset.to_inner_iid(beer_raw_id)
    beer_neighbors = algo.get_neighbors(beer_inner_id, k=10)
    beer_neighbors = (algo.trainset.to_raw_iid(inner_id)
                      for inner_id in beer_neighbors)
    beers_id_recc = []
    beer_name_recc =[]
    beer_style_recc = []
    beer_score_mean = []
    for beer in beer_neighbors:
        beers_id_recc.append(beer)
        beer_name_recc.append(get_beer_name(beer))
        beer_style_recc.append(get_beer_style(beer))
        beer_score_mean.append(get_beer_score_mean(beer))
    beer_reccomendations_df = pd.DataFrame(list(zip(beers_id_recc,beer_name_recc,beer_style_recc,beer_score_mean)),
                                       columns=['beer_id', 'name', 'style', 'score_mean'])
    return beer_reccomendations_df

In [31]:
trainset = algo.trainset

In [ ]:
# Let test some beers!

In [33]:
username = "tamu94"
beer_raw_id = 233 # Corona Light
beers = beers_df['name'].tolist()
predict = algo.predict(username, beer_raw_id)
print (predict)

user: tamu94     item: 233        r_ui = None   est = 1.84   {'was_impossible': False}


In [39]:
username = "tamu94"
beer_raw_id = 412 #Old Rasputin, Russian Imperial Stout
beers = beers_df['name'].tolist()
predict = algo.predict(username, beer_raw_id)
print (predict)

user: tamu94     item: 412        r_ui = None   est = 4.09   {'was_impossible': False}


In [41]:
username = "tamu94"
beer_raw_id = 6 # Turbodog
beers = beers_df['name'].tolist()
predict = algo.predict(username, beer_raw_id)
print (predict)

user: tamu94     item: 6          r_ui = None   est = 3.37   {'was_impossible': False}


In [40]:
# Lets predict the ratings for tamu94 using every beer in the database
tamu_94_predict_df = pd.DataFrame([])
for beer in beers:
    beer_raw_id = get_beer_raw_id(beer)
    predict = algo.predict(username, beer_raw_id)
    tamu_94_predict_df = tamu_94_predict_df.append(pd.DataFrame([predict], columns=['username', 'beer_id', 'r_ui', 'estimate', 'details']))

In [36]:
tamus_picks = pd.merge(tamu_94_predict_df, beers_df, on='beer_id')
tamus_top_10picks = tamus_picks.sort_values(by=['estimate'],ascending= False)[:10]
tamus_top_10picks.head(10)

,username,beer_id,r_ui,estimate,details,inner_ids,score,name,style,brewery_id
993,tamu94,87246,None,4.652219,{'was_impossible': False},756,4.736365,Barrel-Aged Abraxas,American Imperial Stout,25888
859,tamu94,62397,None,4.620314,{'was_impossible': False},751,4.806029,Rare Bourbon County Brand Stout (2010),American Imperial Stout,1146
1078,tamu94,110635,None,4.582417,{'was_impossible': False},757,4.723539,Hunahpu's Imperial Stout - Double Barrel Aged,American Imperial Stout,17981
1027,tamu94,94634,None,4.570309,{'was_impossible': False},371,4.593338,Atrial Rubicite,American Wild Ale,24018
1132,tamu94,131782,None,4.545005,{'was_impossible': False},859,4.669581,Vanilla Rye Bourbon County Brand Stout,American Imperial Stout,1146
774,tamu94,51116,None,4.535621,{'was_impossible': False},956,4.614210,Duck Duck Gooze,Belgian Gueuze,18149
855,tamu94,61100,None,4.529482,{'was_impossible': False},990,4.715469,King Henry,British Barleywine,1146
833,tamu94,57747,None,4.518747,{'was_impossible': False},80,4.665967,Bourbon County Brand Coffee Stout,American Imperial Stout,1146
1061,tamu94,102123,None,4.511719,{'was_impossible': False},792,4.703864,Proprietor's Bourbon County Brand Stout (2013),American Imperial Stout,1146
975,tamu94,84772,None,4.497868,{'was_impossible': False},566,4.546653,West Ashley,American Wild Ale,28019


In [37]:
tamus_bottom_10picks = tamus_picks.sort_values(by=['estimate'],ascending= False)[-10:]
tamus_bottom_10picks.head(10)


,username,beer_id,r_ui,estimate,details,inner_ids,score,name,style,brewery_id
83,tamu94,332,None,1.833855,{'was_impossible': False},531,2.066242,Miller Lite,American Light Lager,105
171,tamu94,837,None,1.770202,{'was_impossible': False},439,1.928670,Coors Light,American Light Lager,306
529,tamu94,20732,None,1.765517,{'was_impossible': False},1137,2.007452,Budweiser Select,American Light Lager,29
137,tamu94,670,None,1.639500,{'was_impossible': False},596,1.916544,Busch Beer,American Adjunct Lager,29
376,tamu94,3734,None,1.619470,{'was_impossible': False},1147,1.795152,Michelob Ultra,American Light Lager,29
228,tamu94,1320,None,1.533590,{'was_impossible': False},79,1.790027,Bud Light,American Light Lager,29
233,tamu94,1341,None,1.509581,{'was_impossible': False},320,1.716901,Keystone Light,American Light Lager,306
244,tamu94,1390,None,1.498563,{'was_impossible': False},563,1.773962,Busch Light,American Light Lager,29
133,tamu94,653,None,1.333921,{'was_impossible': False},1174,1.712514,Natural Ice,American Adjunct Lager,29
255,tamu94,1524,None,1.173005,{'was_impossible': False},467,1.604302,Natural Light,American Light Lager,29
